# Testinnsening av upersonlig skattemelding med næringspesifikasjon
Denne demoen er ment for å vise hvordan flyten for et sluttbrukersystem kan hente et utkast, gjøre endringer, validere/kontrollere det mot Skatteetatens apier, for å sende det inn via Altinn3. 

In [1]:
try: 
    from altinn3 import *
    from skatteetaten_api import main_relay, base64_decode_response, decode_dokument
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from pathlib import Path
except ImportError as e:
    print("Mangler en avhengighet, installer dem via pip")
    !pip install python-jose
    !pip install xmltodict
    !pip install pathlib
    import xmltodict
    from skatteetaten_api import main_relay, base64_decode_response, decode_dokument

    
#hjelpe metode om du vil se en request printet som curl 
def print_request_as_curl(r):
    command = "curl -X {method} -H {headers} -d '{data}' '{uri}'"
    method = r.request.method
    uri = r.request.url
    data = r.request.body
    headers = ['"{0}: {1}"'.format(k, v) for k, v in r.request.headers.items()]
    headers = " -H ".join(headers)
    print(command.format(method=method, headers=headers, data=data, uri=uri))

In [2]:
idporten_header = main_relay()

https://oidc-ver2.difi.no/idporten-oidc-provider/authorize?scope=skatteetaten%3Aformueinntekt%2Fskattemelding%20openid&acr_values=Level3&client_id=8d7adad7-b497-40d0-8897-9a9d86c95306&redirect_uri=http%3A%2F%2Flocalhost%3A12345%2Ftoken&response_type=code&state=5_1plE-p4OF9rQEFdIXkag&nonce=1633506921962760&resource=https%3A%2F%2Fmp-test.sits.no%2Fapi%2Feksterntapi%2Fformueinntekt%2Fskattemelding%2F&code_challenge=-FL7Q5j47R2ZDcJ-3i1g7tijySrDk3hzlzi3LQGk26M=&code_challenge_method=S256&ui_locales=nb
Authorization token received
{'code': ['X2pgIYVfyCWuBnLEeZd0GOFXCQaZc1nsGGemqigmll0'], 'state': ['5_1plE-p4OF9rQEFdIXkag']}
JS : 
{'access_token': 'eyJraWQiOiJjWmswME1rbTVIQzRnN3Z0NmNwUDVGSFpMS0pzdzhmQkFJdUZiUzRSVEQ0IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiItMHp1cDZaNWJqSFB2LUQ1eXpyUWJaQ1JXem4xVnVlRnpTQXdBUTBITjRvPSIsImlzcyI6Imh0dHBzOlwvXC9vaWRjLXZlcjIuZGlmaS5ub1wvaWRwb3J0ZW4tb2lkYy1wcm92aWRlclwvIiwiY2xpZW50X2FtciI6Im5vbmUiLCJwaWQiOiIwMTAxNDcwMTM3NyIsInRva2VuX3R5cGUiOiJCZWFyZXIiLCJjbGllbnRfaWQiOiI4ZDd

127.0.0.1 - - [06/Oct/2021 09:55:45] "GET /token?code=X2pgIYVfyCWuBnLEeZd0GOFXCQaZc1nsGGemqigmll0&state=5_1plE-p4OF9rQEFdIXkag HTTP/1.1" 200 -


# Hent utkast og gjeldende
Her legger vi inn fødselsnummeret vi logget oss inn med, Dersom du velger et annet fødselsnummer så må den du logget på med ha tilgang til skattemeldingen du ønsker å hente

#### Parten nedenfor er brukt for demostrasjon, pass på bruk deres egne testparter når dere tester


01014701377 er daglig leder for 811422762 

In [3]:
s = requests.Session()
s.headers = dict(idporten_header)
fnr="01014701377" #oppdater med test fødselsnummerene du har fått tildelt
orgnr_as = "811423262"

### Utkast

In [4]:
url_utkast = f'https://idporten-api-sbstest.sits.no/api/skattemelding/v2/utkast/2021/{orgnr_as}'

r = s.get(url_utkast)
r

<Response [200]>

### Gjeldende

In [5]:
url_gjeldende = f'https://idporten-api-sbstest.sits.no/api/skattemelding/v2/2021/{orgnr_as}'
r_gjeldende = s.get(url_gjeldende)
r_gjeldende

<Response [200]>

## Fastsatt
Her får en _http 404_ om vedkommende ikke har noen fastsetting, rekjørt denne etter du har sendt inn og fått tilbakemdling i Altinn at den har blitt behandlet, du skal nå ha en fastsatt skattemelding om den har blitt sent inn som Komplett

In [8]:
url_fastsatt = f'https://idporten-api-sbstest.sits.no/api/skattemelding/v2/fastsatt/2021/{orgnr_as}'
r_fastsatt = s.get(url_fastsatt)
r_fastsatt

ReadTimeout: HTTPSConnectionPool(host='mp-test.sits.no', port=443): Read timed out. (read timeout=None)

In [ ]:
r_fastsatt.headers

## Svar fra hent gjeldende 

### Gjeldende dokument referanse: 
I responsen på alle api kallene, være seg utkast/fastsatt eller gjeldene, så følger det med en dokumentreferanse. 
For å kalle valider tjenesten, er en avhengig av å bruke korrekt referanse til gjeldende skattemelding. 

Cellen nedenfor henter ut gjeldende dokumentrefranse printer ut responsen fra hent gjeldende kallet 

In [ ]:
sjekk_svar = r_gjeldende

sme_og_naering_respons = xmltodict.parse(sjekk_svar.text)
skattemelding_base64 = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
sme_base64 = skattemelding_base64["content"]
dokref = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]['skattemeldingdokument']['id']
decoded_sme_xml = decode_dokument(skattemelding_base64)
sme_utkast = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()

print(f"Responsen fra hent gjeldende ser slik ut, gjeldende dokumentrerefanse er {dokref}")
print(sjekk_svar.request.method ,sjekk_svar.request.url)
print(xml.dom.minidom.parseString(sjekk_svar.text).toprettyxml())


In [ ]:
#Legg merge til dokumenter.dokument.type = skattemeldingUpersonlig

with open("../../../src/resources/eksempler/v2/Naeringspesifikasjon-enk-v2_etterBeregning.xml", 'r') as f:
    naering_as_xml = f.read()
    
innsendingstype = "komplett"
naeringsspesifikasjoner_as_b64 = base64.b64encode(naering_as_xml.encode("utf-8"))
naeringsspesifikasjoner_as_b64 = str(naeringsspesifikasjoner_as_b64.decode("utf-8"))
naeringsspesifikasjoner_base64=naeringsspesifikasjoner_as_b64
dok_ref=dokref

valider_konvlutt_v2 = """
<?xml version="1.0" encoding="utf-8" ?>
<skattemeldingOgNaeringsspesifikasjonRequest xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:request:v2">
    <dokumenter>
        <dokument>
            <type>skattemeldingUpersonlig</type>
            <encoding>utf-8</encoding>
            <content>{sme_base64}</content>
        </dokument>
        <dokument>
            <type>naeringsspesifikasjon</type>
            <encoding>utf-8</encoding>
            <content>{naeringsspeifikasjon_base64}</content>
        </dokument>
    </dokumenter>
    <dokumentreferanseTilGjeldendeDokument>
        <dokumenttype>skattemeldingPersonlig</dokumenttype>
        <dokumentidentifikator>{dok_ref}</dokumentidentifikator>
    </dokumentreferanseTilGjeldendeDokument>
    <inntektsaar>2021</inntektsaar>
    <innsendingsinformasjon>
        <innsendingstype>{innsendingstype}</innsendingstype>
        <opprettetAv>TurboSkatt</opprettetAv>
    </innsendingsinformasjon>
</skattemeldingOgNaeringsspesifikasjonRequest>
""".replace("\n","")


naering_enk = valider_konvlutt_v2.format(sme_base64=sme_base64,
                                         naeringsspeifikasjon_base64=naeringsspesifikasjoner_base64,
                                         dok_ref=dok_ref,
                                        innsendingstype=innsendingstype)

# Valider utkast sme med næringsopplysninger

In [ ]:
def valider_sme(payload):
    url_valider = f'https://idporten-api-sbstest.sits.no/api/skattemelding/v2/valider/2021/{orgnr_as}'
    header = dict(idporten_header)
    header["Content-Type"] = "application/xml"
    return s.post(url_valider, headers=header, data=payload)


valider_respons = valider_sme(naering_enk)
resultatAvValidering = xmltodict.parse(valider_respons.text)["skattemeldingOgNaeringsspesifikasjonResponse"]["resultatAvValidering"]

if valider_respons:
    print(resultatAvValidering)
    print()
    print(xml.dom.minidom.parseString(valider_respons.text).toprettyxml())
else:
    print(valider_respons.status_code, valider_respons.headers, valider_respons.text)

# Altinn 3

1. Hent Altinn Token
2. Oppretter en ny instans av skjemaet
3. Laster opp skattemeldingen og næringsopplysninger som et vedlegg

In [ ]:
#1
altinn3_applikasjon = "skd/formueinntekt-skattemelding-v2"
altinn_header = hent_altinn_token(idporten_header)
#2
instans_data = opprett_ny_instans_med_inntektsaar(altinn_header, "2021", fnr, appnavn=altinn3_applikasjon)

## Last opp skattemelding

In [ ]:
#Last opp skattemeldingen
req_send_inn = last_opp_skattedata(instans_data, altinn_header, 
                                   xml=naering_enk, 
                                   data_type="skattemeldingOgNaeringsspesifikasjon",
                                   appnavn=altinn3_applikasjon)
req_send_inn

### Sett statusen klar til bekreftelse.

In [ ]:
req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)

### Åpner skattemelding visningsklient for å se beregnet skattemelding med næringsspesifikasjon.

In [ ]:
url_skattemelding_visning = skattemelding_visning(instans_data, appnavn=altinn3_applikasjon)
print(url_skattemelding_visning)

### Sett statusen klar til henting av skatteetaten.

req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)
req_bekreftelse

### Framtidig: Sjekk status på altinn3 instansen om skatteetaten har hentet instansen. 

### Se innsending i Altinn

Ta en slurk av kaffen og klapp deg selv på ryggen, du har nå sendt inn, la byråkratiet gjøre sin ting... og det tar litt tid. Pt så sjekker skatteeaten hos Altinn3 hvert 5 min om det har kommet noen nye innsendinger. 